In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gc
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('./Binance_BTCUSDT.csv', date_parser = True)
df = df.reindex(-np.sort(-np.arange(len(df))))
len(df)

41240

In [3]:
#擷取⽇期及收盤價
df2 = df.loc[:, ('date','close', 'open','high','low','Volume BTC')] ; del df ; gc.collect()
#切出⽇期時間欄位，符號統⼀
df2['NewDate'] = df2.loc[:, 'date'].str.slice(start = 0, stop = 10).str.replace('-', '/').str.replace('/0', '/').str.rsplit(pat=' ', n=1).apply(lambda x : x[0])
#判斷⽇期欄位格式
df2['AMPM'] = df2.loc[:, 'date'].str.contains(pat = 'AM|PM')
#依⽇期欄位格式切分兩分DF，有AMPM的在df3，正常格式的在df4
df3 = df2[df2['AMPM'] == 1]
df4 = df2[df2['AMPM'] == 0]
del df2 ; gc.collect()


33

In [4]:
#切出⽇期欄位中的⼩時
df3['Hour'] = df3.loc[:, 'date'].str.slice(start = 10, stop = 13).str.replace(':', '').apply(lambda x : pd.to_numeric(x))
df3['AMPM'] = df3.loc[:, 'date'].str.slice(start = 14, stop = 16).apply(lambda x: 12 if (x == 'PM') else 0)
df3['24Hr'] = (df3['Hour'] + df3['AMPM'])
#轉換為24⼩時制
df3['24Hr'] = df3['24Hr'].apply(lambda x: (x == 24 and 12) or (x == 12 and '00') or x)
df3['24Hr'] = df3['24Hr'].apply(lambda x: '0'+str(x) if len(str(x)) == 1 else str(x))

#轉為時間格式
df3['NewDateTime'] = (df3['NewDate'] + ' ' +df3['24Hr'])
df3['NewDateTime'] = df3['NewDateTime'].apply(lambda x: pd.to_datetime(x, format = "%Y/%m/%d %H"))
#移除重複
df3 = df3[['close', 'open','high','low','Volume BTC', 'NewDateTime','NewDate']].drop_duplicates()
df3.head()

,close,open,high,low,Volume BTC,NewDateTime,NewDate
41239,4308.83,16199.91,16199.91,4261.32,44.5100,2017-08-17 04:00:00,2017/8/17
41238,4315.32,4308.83,4328.69,4291.37,23.2300,2017-08-17 05:00:00,2017/8/17
41237,4324.35,4315.32,4345.45,4309.37,7.2300,2017-08-17 06:00:00,2017/8/17
41236,4349.99,4324.35,4349.99,4287.41,4.4400,2017-08-17 07:00:00,2017/8/17
41235,4360.69,4349.99,4377.85,4333.32,0.9499,2017-08-17 08:00:00,2017/8/17


In [5]:
#轉為時間格式並移除重複
df4['NewDateTime'] = df4['date'].apply(lambda x: pd.to_datetime(x, format = "%Y/%m/%d %H:%M"))
df4 = df4[['close', 'open','high','low','Volume BTC', 'NewDateTime','NewDate']].drop_duplicates()
df4.head()

,close,open,high,low,Volume BTC,NewDateTime,NewDate
15303,11801.17,11783.31,11820.61,11731.71,2194.598589,2020-08-01 23:00:00,2020/8/1
15301,11782.63,11801.17,11857.00,11740.00,2872.638184,2020-08-02 00:00:00,2020/8/2
15299,11942.55,11782.63,11942.55,11772.13,3707.136801,2020-08-02 01:00:00,2020/8/2
15297,11928.58,11942.54,11965.01,11892.88,3785.415559,2020-08-02 02:00:00,2020/8/2
15295,12000.00,11928.40,12000.00,11870.00,3746.422743,2020-08-02 03:00:00,2020/8/2


In [6]:
#合併df3、df4，並移除重複
df5 = pd.concat([df3, df4]) ; df6 = df5.drop_duplicates()
#確認刪除多少資料
print(len(df5), len(df6))
del (df3, df4, df5)
gc.collect()


41240 41240


0

In [7]:
#投報率
df6['returns']=df6['close']/df6['close'].shift(1)
df6['log_returns']=np.log(df6['returns'])
df6 = df6[df6['returns'].notna()]
# df6.dropna()
df6.head()

,close,open,high,low,Volume BTC,NewDateTime,NewDate,returns,log_returns
41238,4315.32,4308.83,4328.69,4291.37,23.2300,2017-08-17 05:00:00,2017/8/17,1.001506,0.001505
41237,4324.35,4315.32,4345.45,4309.37,7.2300,2017-08-17 06:00:00,2017/8/17,1.002093,0.002090
41236,4349.99,4324.35,4349.99,4287.41,4.4400,2017-08-17 07:00:00,2017/8/17,1.005929,0.005912
41235,4360.69,4349.99,4377.85,4333.32,0.9499,2017-08-17 08:00:00,2017/8/17,1.002460,0.002457
41234,4444.00,4360.69,4445.78,4360.00,10.7600,2017-08-17 09:00:00,2017/8/17,1.019105,0.018925


In [8]:
#依照年份及週次分組
df6['Year'] = df6.loc[:, 'NewDateTime'].dt.year
df6['Week'] = df6.loc[:, 'NewDateTime'].dt.isocalendar().week
test = np.array(df6.groupby(['Year', 'Week']), dtype = object).T
#排除⻑度不為168的array
Array = np.empty([1, 168])
for i in range(0, len(np.array(test[1]))):
    if len(np.array(test[1][i]['close'])) == 168 :
        Array = np.append(Array, np.array(test[1][i]['close']).reshape(1, -1), axis = 0)

AttributeError: 'DatetimeProperties' object has no attribute 'isocalendar'

In [9]:
df6.head()

,close,open,high,low,Volume BTC,NewDateTime,NewDate,returns,log_returns,Year
41238,4315.32,4308.83,4328.69,4291.37,23.2300,2017-08-17 05:00:00,2017/8/17,1.001506,0.001505,2017
41237,4324.35,4315.32,4345.45,4309.37,7.2300,2017-08-17 06:00:00,2017/8/17,1.002093,0.002090,2017
41236,4349.99,4324.35,4349.99,4287.41,4.4400,2017-08-17 07:00:00,2017/8/17,1.005929,0.005912,2017
41235,4360.69,4349.99,4377.85,4333.32,0.9499,2017-08-17 08:00:00,2017/8/17,1.002460,0.002457,2017
41234,4444.00,4360.69,4445.78,4360.00,10.7600,2017-08-17 09:00:00,2017/8/17,1.019105,0.018925,2017


In [10]:
# 加入SOPR
df_SOPR = pd.read_csv('./sopr-btc-24h.csv', date_parser = True)
df_SOPR = df_SOPR.reindex(-np.sort(-np.arange(len(df_SOPR))))
df_SOPR.tail()
df_SOPR2 = df_SOPR.loc[:, ('t','v')] ; del df_SOPR ; gc.collect()
#切出⽇期時間欄位，符號統⼀
df_SOPR2['NewDate'] = df_SOPR2.loc[:, 't'].str.slice(start = 0, stop = 10).str.replace('-', '/').str.replace('/0', '/').str.rsplit(pat=' ', n=1).apply(lambda x : x[0])
df_SOPR2 = df_SOPR2[['v','NewDate']].drop_duplicates()
merge_SOPR = pd.merge(df6,df_SOPR2, on='NewDate')
merge_SOPR = merge_SOPR.rename(columns = {"v":"SOPR"})
len(merge_SOPR)


41239

In [11]:
merge_SOPR['Hour'] = merge_SOPR.loc[:, 'NewDateTime'].dt.hour
merge_SOPR['three'] = merge_SOPR['Hour'].apply(lambda x: 0 if x<8 else (8 if x<16 else 16))
merge_SOPR['three'] = merge_SOPR['Hour'].apply(lambda x: '0' if x<8 else ('8' if x<16 else '16'))
merge_SOPR['8h'] =  merge_SOPR["NewDate"] + ":" + merge_SOPR["three"]
merge_SOPR= merge_SOPR.drop(['Hour', 'three'], axis=1)
merge_SOPR.tail(504)

,close,open,high,low,Volume BTC,NewDateTime,NewDate,returns,log_returns,Year,SOPR,8h
40735,48818.89,48644.12,48847.14,48600.00,742.92826,2021-12-22 01:00:00,2021/12/22,1.003593,0.003586,2021,1.001202,2021/12/22:0
40736,49326.50,48818.90,49576.13,48727.47,1987.96756,2021-12-22 02:00:00,2021/12/22,1.010398,0.010344,2021,1.001202,2021/12/22:0
40737,49249.00,49327.24,49446.35,49141.15,892.82053,2021-12-22 03:00:00,2021/12/22,0.998429,-0.001572,2021,1.001202,2021/12/22:0
40738,49312.40,49249.00,49347.90,49081.78,1026.27797,2021-12-22 04:00:00,2021/12/22,1.001287,0.001287,2021,1.001202,2021/12/22:0
40739,48964.88,49312.40,49331.90,48914.07,948.35712,2021-12-22 05:00:00,2021/12/22,0.992953,-0.007072,2021,1.001202,2021/12/22:0
...,...,...,...,...,...,...,...,...,...,...,...,...
41234,42800.38,42972.04,43095.26,42692.19,1219.60178,2022-01-11 20:00:00,2022/1/11,0.996005,-0.004003,2022,0.999457,2022/1/11:16
41235,42659.20,42797.62,42823.69,42643.74,702.10380,2022-01-11 21:00:00,2022/1/11,0.996701,-0.003304,2022,0.999457,2022/1/11:16
41236,42713.13,42664.71,42776.14,42597.41,561.85993,2022-01-11 22:00:00,2022/1/11,1.001264,0.001263,2022,0.999457,2022/1/11:16
41237,42729.29,42713.12,42886.28,42633.97,681.14201,2022-01-11 23:00:00,2022/1/11,1.000378,0.000378,2022,0.999457,2022/1/11:16


In [12]:
# 加入Funding rate
df_fr = pd.read_csv('./Funding Rate History_BTCUSDT.csv', date_parser = True)
df_fr = df_fr.reindex(-np.sort(-np.arange(len(df_fr))))
df_fr2 = df_fr.loc[:, ('Time','Funding Rate')] ; del df_fr ; gc.collect()
df_fr2['NewDate'] = df_fr2.loc[:, 'Time'].str.slice(start = 0, stop = 10).str.replace('-', '/').str.replace('/0', '/').str.rsplit(pat=' ', n=1).apply(lambda x : x[0])
# df_fr2 = df_fr2[['Funding Rate','NewDate']].drop_duplicates()


df_fr2['NewDateTime'] = df_fr2['Time'].apply(lambda x: pd.to_datetime(x, format = "%Y/%m/%d %H:%M"))
df_fr2['Hour'] = df_fr2.loc[:, 'NewDateTime'].dt.hour
df_fr2['three'] = df_fr2['Hour'].apply(lambda x: '0' if x<8 else ('8' if x<16 else '16'))
df_fr2['8h'] =  df_fr2["NewDate"] + ":" + df_fr2["three"]
df_fr2= df_fr2.drop(['Time','NewDate','NewDateTime','Hour', 'three'], axis=1)
df_fr2['fundingRate'] = df_fr2.loc[:, 'Funding Rate'].str.replace('%', '')
df_fr2 = df_fr2.drop(['Funding Rate'], axis=1)

merge_fr = pd.merge(merge_SOPR,df_fr2, on='8h')
merge_fr.tail(504)

,close,open,high,low,Volume BTC,NewDateTime,NewDate,returns,log_returns,Year,SOPR,8h,fundingRate
22632,48818.89,48644.12,48847.14,48600.00,742.92826,2021-12-22 01:00:00,2021/12/22,1.003593,0.003586,2021,1.001202,2021/12/22:0,0.0100
22633,49326.50,48818.90,49576.13,48727.47,1987.96756,2021-12-22 02:00:00,2021/12/22,1.010398,0.010344,2021,1.001202,2021/12/22:0,0.0100
22634,49249.00,49327.24,49446.35,49141.15,892.82053,2021-12-22 03:00:00,2021/12/22,0.998429,-0.001572,2021,1.001202,2021/12/22:0,0.0100
22635,49312.40,49249.00,49347.90,49081.78,1026.27797,2021-12-22 04:00:00,2021/12/22,1.001287,0.001287,2021,1.001202,2021/12/22:0,0.0100
22636,48964.88,49312.40,49331.90,48914.07,948.35712,2021-12-22 05:00:00,2021/12/22,0.992953,-0.007072,2021,1.001202,2021/12/22:0,0.0100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23131,42800.38,42972.04,43095.26,42692.19,1219.60178,2022-01-11 20:00:00,2022/1/11,0.996005,-0.004003,2022,0.999457,2022/1/11:16,0.0073
23132,42659.20,42797.62,42823.69,42643.74,702.10380,2022-01-11 21:00:00,2022/1/11,0.996701,-0.003304,2022,0.999457,2022/1/11:16,0.0073
23133,42713.13,42664.71,42776.14,42597.41,561.85993,2022-01-11 22:00:00,2022/1/11,1.001264,0.001263,2022,0.999457,2022/1/11:16,0.0073
23134,42729.29,42713.12,42886.28,42633.97,681.14201,2022-01-11 23:00:00,2022/1/11,1.000378,0.000378,2022,0.999457,2022/1/11:16,0.0073


In [17]:
# 加入google trend
df_GT = pd.read_csv('./googleTrend.csv', date_parser = True)
df_GT = df_GT.reindex(-np.sort(-np.arange(len(df_GT))))
df_GT = df_GT[df_GT['date']< '2022-01-12'].copy()
df_GT[384:408]
df_GT2 = df_GT.loc[:, ('date','bitcoin')] ; del df_GT ; gc.collect()

df_GT2['newdate']=df_GT2.loc[:, 'date'].str.slice(start = 0, stop = 19)
#切出⽇期時間欄位，符號統⼀
df_GT2['NewDateTime'] = df_GT2['newdate'].apply(lambda x: pd.to_datetime(x, format = "%Y/%m/%d %H"))
df_GT2 = df_GT2.drop(['date','newdate'], axis=1)
df_GT2.head(511)
merge_GT = pd.merge(merge_fr,df_GT2, on='NewDateTime')
merge_GT = merge_GT.rename(columns = {"bitcoin":"googleTrend"})
merge_GT.tail(504)


,close,open,high,low,Volume BTC,NewDateTime,NewDate,returns,log_returns,Year,SOPR,8h,fundingRate,googleTrend
19796,48818.89,48644.12,48847.14,48600.00,742.92826,2021-12-22 01:00:00,2021/12/22,1.003593,0.003586,2021,1.001202,2021/12/22:0,0.0100,42
19797,49326.50,48818.90,49576.13,48727.47,1987.96756,2021-12-22 02:00:00,2021/12/22,1.010398,0.010344,2021,1.001202,2021/12/22:0,0.0100,57
19798,49326.50,48818.90,49576.13,48727.47,1987.96756,2021-12-22 02:00:00,2021/12/22,1.010398,0.010344,2021,1.001202,2021/12/22:0,0.0100,38
19799,49249.00,49327.24,49446.35,49141.15,892.82053,2021-12-22 03:00:00,2021/12/22,0.998429,-0.001572,2021,1.001202,2021/12/22:0,0.0100,64
19800,49312.40,49249.00,49347.90,49081.78,1026.27797,2021-12-22 04:00:00,2021/12/22,1.001287,0.001287,2021,1.001202,2021/12/22:0,0.0100,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20295,42972.05,42754.99,43086.70,42595.68,1440.44569,2022-01-11 19:00:00,2022/1/11,1.005077,0.005064,2022,0.999457,2022/1/11:16,0.0073,48
20296,42800.38,42972.04,43095.26,42692.19,1219.60178,2022-01-11 20:00:00,2022/1/11,0.996005,-0.004003,2022,0.999457,2022/1/11:16,0.0073,46
20297,42659.20,42797.62,42823.69,42643.74,702.10380,2022-01-11 21:00:00,2022/1/11,0.996701,-0.003304,2022,0.999457,2022/1/11:16,0.0073,45
20298,42713.13,42664.71,42776.14,42597.41,561.85993,2022-01-11 22:00:00,2022/1/11,1.001264,0.001263,2022,0.999457,2022/1/11:16,0.0073,43


In [18]:
# save updated dataset
merge_GT = merge_GT.drop(['NewDate','Year','8h'], axis=1)
merge_GT.tail(504)



,close,open,high,low,Volume BTC,NewDateTime,returns,log_returns,SOPR,fundingRate,googleTrend
19796,48818.89,48644.12,48847.14,48600.00,742.92826,2021-12-22 01:00:00,1.003593,0.003586,1.001202,0.0100,42
19797,49326.50,48818.90,49576.13,48727.47,1987.96756,2021-12-22 02:00:00,1.010398,0.010344,1.001202,0.0100,57
19798,49326.50,48818.90,49576.13,48727.47,1987.96756,2021-12-22 02:00:00,1.010398,0.010344,1.001202,0.0100,38
19799,49249.00,49327.24,49446.35,49141.15,892.82053,2021-12-22 03:00:00,0.998429,-0.001572,1.001202,0.0100,64
19800,49312.40,49249.00,49347.90,49081.78,1026.27797,2021-12-22 04:00:00,1.001287,0.001287,1.001202,0.0100,60
...,...,...,...,...,...,...,...,...,...,...,...
20295,42972.05,42754.99,43086.70,42595.68,1440.44569,2022-01-11 19:00:00,1.005077,0.005064,0.999457,0.0073,48
20296,42800.38,42972.04,43095.26,42692.19,1219.60178,2022-01-11 20:00:00,0.996005,-0.004003,0.999457,0.0073,46
20297,42659.20,42797.62,42823.69,42643.74,702.10380,2022-01-11 21:00:00,0.996701,-0.003304,0.999457,0.0073,45
20298,42713.13,42664.71,42776.14,42597.41,561.85993,2022-01-11 22:00:00,1.001264,0.001263,0.999457,0.0073,43


In [19]:
merge_GT.to_csv('bitcoin_data.csv')